In [43]:
# Reg fetch new batch of features and compute predictions and save to feature store
# 

In [44]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [45]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [46]:
from src.inference import get_feature_store
from datetime import datetime, timedelta
import pandas as pd  

# Get the current datetime64[us, Etc/UTC]  
current_date = pd.Timestamp.now(tz='Etc/UTC')
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1)),
)
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)

from src.data_utils import transform_ts_data_info_features
features = transform_ts_data_info_features(ts_data, window_size=24 * 28, step_size=23)

2025-03-06 00:26:52,526 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-06 00:26:52,530 INFO: Initializing external client
2025-03-06 00:26:52,531 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-06 00:26:53,463 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215669
Fetching data from 2025-02-05 05:26:52.526978+00:00 to 2025-03-06 04:26:52.526978+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (4.74s) 


In [47]:
current_date

Timestamp('2025-03-06 05:26:52.526978+0000', tz='Etc/UTC')

In [48]:
current_date.ceil('h')

Timestamp('2025-03-06 06:00:00+0000', tz='Etc/UTC')

In [49]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

2025-03-06 00:27:06,828 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-06 00:27:06,833 INFO: Initializing external client
2025-03-06 00:27:06,834 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-06 00:27:07,687 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215669


In [50]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,0,0,217,2025-02-27 00:00:00
1,0,0,0,0,0,0,0,0,0,0,...,16,12,1,8,2,2,35,11,261,2025-02-09 19:00:00
2,0,0,0,0,0,0,0,0,0,0,...,0,6,6,0,0,1,1,0,65,2025-02-10 23:00:00
3,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,2,0,69,2025-02-09 13:00:00
4,0,0,0,0,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,155,2025-02-22 23:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,0,0,0,0,0,0,0,0,0,0,...,2,0,0,1,5,2,0,3,49,2025-02-24 16:00:00
247,0,0,0,0,0,0,0,0,0,0,...,3,0,1,0,1,0,0,1,56,2025-02-18 07:00:00
248,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,202,2025-02-16 22:00:00
249,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,153,2025-02-06 07:00:00


In [51]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

In [52]:
predictions["pickup_hour"] = current_date.ceil('h')
predictions

,pickup_location_id,predicted_demand,pickup_hour
0,217,0.0,2025-03-06 06:00:00+00:00
1,261,5.0,2025-03-06 06:00:00+00:00
2,65,1.0,2025-03-06 06:00:00+00:00
3,69,1.0,2025-03-06 06:00:00+00:00
4,155,0.0,2025-03-06 06:00:00+00:00
...,...,...,...
246,49,1.0,2025-03-06 06:00:00+00:00
247,56,0.0,2025-03-06 06:00:00+00:00
248,202,0.0,2025-03-06 06:00:00+00:00
249,153,0.0,2025-03-06 06:00:00+00:00


In [53]:
from src.inference import get_feature_store

feature_group = get_feature_store().get_or_create_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTION,
    version=1,
    description="Predictions from LGBM Model",
    primary_key=["pickup_location_id", "pickup_hour"],
    event_time="pickup_hour",
)

2025-03-06 00:27:10,393 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-06 00:27:10,398 INFO: Initializing external client
2025-03-06 00:27:10,398 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-06 00:27:11,102 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215669


In [54]:
feature_group.insert(predictions, write_options={"wait_for_job": False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1215669/fs/1202311/fg/1402134


Uploading Dataframe: 100.00% |██████████| Rows 251/251 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: taxi_hourly_model_prediction_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1215669/jobs/named/taxi_hourly_model_prediction_1_offline_fg_materialization/executions


(Job('taxi_hourly_model_prediction_1_offline_fg_materialization', 'SPARK'),
 None)